# IMDB 영화평 감성분석

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('../00. data/IMDB/labeledTrainData.tsv', header=0, sep='\t', quoting=3)
df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [4]:
df['review'][0]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

In [5]:
# <br /> 태그 변환
df['review'] = df.review.str.replace('<br / >', ' ')

In [6]:
# 영어 이외의 문자는 공백으로 변환
import re
df['review'] = df.review.apply(lambda x: re.sub('[^a-zA-Z]', ' ', x))

In [8]:
from sklearn.model_selection import train_test_split

feature_df = df.drop(['id', 'sentiment'], axis=1, inplace=False)
X_train, X_test, y_train, y_test = train_test_split(feature_df, df.sentiment, test_size=0.3, random_state=156)
X_train.shape, X_test.shape

((17500, 1), (7500, 1))

In [9]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

- CountVectorizer

In [12]:
cv = CountVectorizer(stop_words='english', ngram_range=(1, 2))
cv.fit(X_train.review) # df를 넣었기 때문에 지정
X_train_count = cv.transform(X_train.review) 
X_test_count = cv.transform(X_test.review)

In [13]:
lr = LogisticRegression(C=10)
lr.fit(X_train_count, y_train)
pred = lr.predict(X_test_count)
accuracy_score(y_test, pred)

0.8865333333333333

- TfidfVectorizer

In [15]:
tv = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))
tv.fit(X_train.review)
X_train_tfidf = tv.transform(X_train.review)
X_test_tfidf = tv.transform(X_test.review)

In [16]:
lr.fit(X_train_tfidf, y_train)
pred = lr.predict(X_test_tfidf)
accuracy_score(y_test, pred)

0.8932

### 모델 저장하고 불러오기

In [17]:
import joblib
joblib.dump(tv, 'model/imdb_vect.pkl')
joblib.dump(lr, 'model/imdb_lr.pkl')

['model/imdb_lr']

In [18]:
del tv
del lr

In [19]:
new_vect = joblib.load('model/imdb_vect.pkl')
new_lr = joblib.load('model/imdb_lr.pkl')

In [20]:
new_X_train = new_vect.transform(X_train.review)
new_X_test = new_vect.transform(X_test.review)

In [21]:
pred = new_lr.predict(new_X_test)
accuracy_score(y_test, pred)

0.8932

### Pipeline을 써서 학습/예측/평가

In [22]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('count_vect', CountVectorizer(stop_words='english', ngram_range=(1, 2))),
    ('lr_clf', LogisticRegression(C=10))
])
pipeline.fit(X_train.review, y_train)
pred = pipeline.predict(X_test.review)
acc = accuracy_score(y_test, pred)
print(f'Count Vectorize + Logistic Regression 정확도: {acc:.4f}')

Count Vectorize + Logistic Regression 정확도: 0.8865


In [23]:
joblib.dump(pipeline, 'model/pipeline.pkl')

['model/pipeline.pkl']

In [24]:
new_pipe = joblib.load('model/pipeline.pkl')

In [25]:
pred = new_pipe.predict(X_test.review)
acc = accuracy_score(y_test, pred)
print(f'Count Vectorize + Logistic Regression 정확도: {acc:.4f}')

Count Vectorize + Logistic Regression 정확도: 0.8865


### 하이퍼 파라미터 튜닝

In [26]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('tv', TfidfVectorizer(stop_words='english')),
    ('lr', LogisticRegression())
])

In [33]:
params = {
    'tv__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'tv__max_df': [700, 900, 1000],
    'lr__C': [10, 20, 30]
}

In [34]:
from sklearn.model_selection import GridSearchCV
grid_pipe = GridSearchCV(pipeline, param_grid=params, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
grid_pipe.fit(X_train.review, y_train)
print(grid_pipe.best_params_, grid_pipe.best_score_)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  7.6min
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed: 27.0min finished
{'lr__C': 30, 'tv__max_df': 1000, 'tv__ngram_range': (1, 2)} 0.8894285714285715


In [35]:
pred = grid_pipe.predict(X_test.review)
accuracy_score(y_test, pred)

0.8949333333333334